In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import konlpy
import os

import warnings

import nltk
from nltk.corpus import stopwords
from collections import Counter
from konlpy.tag import Twitter
twitter = Twitter()
warnings.filterwarnings("ignore")


#df_raw = pd.read_excel('./DataFrame/esence_perfect.xlsx')

#df_columns = df.columns.values



C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## 데이터 프레임 합치기

In [ ]:
# df_cream2 = df_total[:7544].drop(columns='Unnamed: 0')
# df_essence2 = df_total[7544:13981].drop(columns='Unnamed: 0')
# df_oil2 = df_total[13981:14981].drop(columns='Unnamed: 0')
# df_lotion2 = df_total[14981:18387].drop(columns='Unnamed: 0')
# df_mist2 = df_total[18387:21387].drop(columns='Unnamed: 0')
# df_skin_toner2 = df_total[21387:].drop(columns='Unnamed: 0')

In [13]:
# 스킨 케어 데이터 가져오기.
def call_skin_care() :
    raw_list = []
    for dirname, _, filenames in os.walk('./DataFrame/스킨케어_이미지포함'):
        for filename in filenames:
            print(os.path.join(dirname, filename))
            raw_list.append(pd.read_excel(os.path.join(dirname, filename)))

    raw_data = pd.concat(raw_list)
    return raw_list

In [14]:
# 스킨케어에 있는 data_frame 종류 보기
cosmetics = call_skin_care()

./DataFrame/스킨케어_이미지포함\cream_complete.xlsx
./DataFrame/스킨케어_이미지포함\essence_complete.xlsx
./DataFrame/스킨케어_이미지포함\lotion_complete.xlsx
./DataFrame/스킨케어_이미지포함\mist_complete.xlsx
./DataFrame/스킨케어_이미지포함\oil_complete.xlsx
./DataFrame/스킨케어_이미지포함\skin_toner_complete.xlsx


In [15]:
len(cosmetics)

6

In [16]:
cosmetics[4]['price_num'].value_counts()

 16,300    100
 38,000    100
 20,000    100
 36,000     50
 27,000     50
 43,000     50
 68,000     50
 39,000     50
 28,000     50
 26,900     50
 14,000     50
 42,000     50
 6,500      50
 17,000     50
 12,100     50
 30,000     50
 49,000     50
Name: price_num, dtype: int64

In [17]:
# 데이터 전처리_part1

def data_preprocess(df_list):
    dataframe_list = []
    for df in df_list:
        for j in range(len(df['price_num'])):
            if df['price_num'][j] == ' 가격미정':
                df['price_num'][j] = None
                
        # 성분정보 결측치 처리
        df['ingredient'] = df['ingredient'].fillna('성분정보없음')
        # 제품 가격 결측치 처리
        df['price_num'] = df['price_num'].apply(
            lambda x : int(x.replace(',','').replace(' ','')) if (bool(x) == True) else x)
        df['price_num'] = df['price_num'].fillna(df['price_num'].mean())
        df['danger_num'] = df['danger_num'].apply(lambda x : 0 if (x == '없음') else x)
        df['danger_num'] = df['danger_num'].apply(lambda x : int(x))
        # 가격,위험성분 column scoring and Scaler
        min_max_sc = MinMaxScaler()
        df['price_num_score'] = min_max_sc.fit_transform(df[['price_num']])
        df['danger_num_score'] = min_max_sc.fit_transform(df[['danger_num']])
        df['price_num'] = df['price_num'].astype(int)
        # 리뷰 결측치 처리
        df['review'] = df['review'].fillna('리뷰없음')
        
        dataframe_list.append(df)
    return dataframe_list

In [18]:
cosmetics_list = data_preprocess(cosmetics)

In [19]:
# 데이터 전처리_part2
# 리뷰 토큰화 - 향후 장고에서 제품 추천 알고리즘 적용 시 데이터 처리 속도 향상을 위하여 사전 작업 진행.
okt = konlpy.tag.Okt()
def tokenize(df_list):
    dataframe_list = []
    for df in df_list:
        col_to = []
        for item in df['review']:
            col_to.append(okt.morphs(item))
        review_to = []
        for j in col_to:
            review_to.append(' '.join(j))

        df['review'] = review_to
        dataframe_list.append(df)
    return dataframe_list

In [20]:
cosmetics_li = tokenize(cosmetics_list)

In [22]:
cosmetics_li[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7451 entries, 0 to 7450
Data columns (total 21 columns):
brand               7451 non-null object
name                7451 non-null object
bottle              7451 non-null object
price_num           7451 non-null int32
star                7451 non-null float64
product_info        7451 non-null object
product_type        7451 non-null object
ingredient          7451 non-null object
danger_num          7451 non-null int64
rank_num            7451 non-null int64
age                 7451 non-null int64
skintype            7451 non-null object
gender              7451 non-null object
rating              7451 non-null int64
review              7451 non-null object
offer               7451 non-null int64
소분류                 7451 non-null object
대분류                 7451 non-null object
crim_image_link     7451 non-null object
price_num_score     7451 non-null float64
danger_num_score    7451 non-null float64
dtypes: float64(3), int32(1), int64

In [25]:
# 데이터 전처리_part3
# product_id열 추가
def id_in(df_list):
    dataframe_list = []
    for df in df_list:
        df['product_id'] = None
        df['product_id'][0] = 1
        for i in range(1,len(df['name'])):
            if df['product_id'][i-1] and df['name'][i] == df['name'][i-1]:
                df['product_id'][i] = df['product_id'][i-1]

            else:
                df['product_id'][i] = (df['product_id'][i-1]) +1
        df['review'] = df['review'].fillna('리뷰없음')
        dataframe_list.append(df)
    return dataframe_list

In [26]:
cosmetics_list_final = id_in(cosmetics_li)

In [ ]:
# for i in range(1,len(df_rev1['name'])):
#     if df_rev1['product_id'][i-1] and df_rev1['name'][i] == df_rev1['name'][i-1]:
#         df_rev1['product_id'][i] = df_rev1['product_id'][i-1]
        
#     else:
#         df_rev1['product_id'][i] = (df_rev1['product_id'][i-1]) +1

In [33]:
cosmetics_list_final[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3692 entries, 0 to 3691
Data columns (total 22 columns):
brand                    3692 non-null object
name                     3692 non-null object
bottle                   3692 non-null object
price_num                3692 non-null int32
star                     3692 non-null float64
product_info             3692 non-null object
product_type             3692 non-null object
ingredient               3692 non-null object
danger_num               3692 non-null int64
rank_num                 3692 non-null int64
age                      3692 non-null int64
skintype                 3692 non-null object
gender                   3692 non-null object
rating                   3692 non-null int64
review                   3692 non-null object
offer                    3692 non-null int64
소분류                      3692 non-null object
대분류                      3692 non-null object
skin_tonur_image_link    3692 non-null object
price_num_score         

In [34]:
cosmetics_list_final[0].to_excel('cream_complate.xlsx', encoding='euc-kr', index = False)
cosmetics_list_final[1].to_excel('essence_complate.xlsx', encoding='euc-kr', index = False)
cosmetics_list_final[2].to_excel('oil_complate.xlsx', encoding='euc-kr', index = False)
cosmetics_list_final[3].to_excel('lotion_complate.xlsx', encoding='euc-kr', index = False)
cosmetics_list_final[4].to_excel('mist_complate.xlsx', encoding='euc-kr', index = False)
cosmetics_list_final[5].to_excel('skin_toner_complate.xlsx', encoding='euc-kr', index = False)

In [35]:
df_raw = pd.read_excel('./DataFrame/스킨케어_fin/cream_complete.xlsx')

In [37]:
df_raw['review'] = df_raw['review'].fillna('리뷰없음')

In [38]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7451 entries, 0 to 7450
Data columns (total 22 columns):
brand               7451 non-null object
name                7451 non-null object
bottle              7451 non-null object
price_num           7451 non-null int64
star                7451 non-null float64
product_info        7451 non-null object
product_type        7451 non-null object
ingredient          7451 non-null object
danger_num          7451 non-null int64
rank_num            7451 non-null int64
age                 7451 non-null int64
skintype            7451 non-null object
gender              7451 non-null object
rating              7451 non-null int64
review              7451 non-null object
offer               7451 non-null int64
소분류                 7451 non-null object
대분류                 7451 non-null object
crim_image_link     7451 non-null object
price_num_score     7451 non-null float64
danger_num_score    7451 non-null float64
product_id          7451 non-null i

In [39]:
df_raw.to_excel('cream_complate.xlsx', encoding='euc-kr', index = False)

## 태그 중복값 확인

In [ ]:
# product = df['product_type'].value_counts()

# product

# set(product.to_dict().keys())

# len(set(product.to_dict().keys()))

# mylist = list(set(product.to_dict().keys()))

# newlist = [word for line in mylist for word in line.split()]

# print(len(list(set(newlist))))

## tfidf, count_vector 함수

In [ ]:
# TF-IDF 적용
def tfidf_vect_func(df, target_col, target_row):

    tfidf_vect = TfidfVectorizer()
    feature_vect = tfidf_vect.fit_transform(df[target_col])
    
    #코사인 유사도 분석
    similarity_simple_pair = cosine_similarity(feature_vect[target_row], feature_vect)
    result_list = similarity_simple_pair.tolist()[0]
    df[f'{target_col}_result'] = result_list
    
    return df

In [ ]:
#count_vectorizer 적용
def count_vect_func(df,target_col, target_row):
    count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
    feature_vect = count_vect_category.fit_transform(df[target_col])
    
    #코사인 유사도 분석
    similarity_simple_pair = cosine_similarity(feature_vect[target_row], feature_vect)
    result_list = similarity_simple_pair.tolist()[0]
    df[f'{target_col}_result'] = result_list
    
    return df

## 한 행에 한 리뷰, top 50 중 가장 많은 빈도수 기준 추천

In [ ]:
cosmetics_list[0]

In [ ]:
# 고객의 리뷰, 제품 태그를 입력받아 유사도 분석 진행
def tfidf_count_result(df,review_col,tag_col,problem,hashtag):

    df = df.append(pd.Series(),ignore_index= True)
    last_index = df.iloc[[-1]].index[0]

    df.at[last_index, review_col] = problem  #input('피부 고민을 말씀해주세요. : ')\
    df.at[last_index,tag_col] = hashtag      #input('태그를 입력해주세요 : ')

    df = tfidf_vect_func(df, review_col, last_index)
    df = count_vect_func(df, tag_col, last_index)
    
    return df

In [ ]:
df = tfidf_count_result(cosmetics_list[0],'review','product_type')

In [ ]:
df

In [ ]:
# df['result_sum'] = df['review_result']+(df['product_type_result']*0.5)

## 위에서 20개 잘라주고 중복 값 더해주고 sort (중복값 가중치 주기) 

In [ ]:
def sort_review(df, num):
    #tag 가중치
    df['review_result'] = df['review_result']+(df['product_type_result']*0.05)

    #result 값으로 sort후 잘라주기
    df_review_sort = df.sort_values(by='review_result',ascending=False).copy()
    df_review_sort = df_review_sort[1:]
    df_review_cut = df_review_sort[:num]

    #중복값 합치기
    df_review_cut2 = df_review_cut[['name','review_result']]
    df_review_cut2 = df_review_cut2.groupby('name')['review_result'].apply(lambda x: x.sum())
    df_review_cut2 = pd.DataFrame(df_review_cut2)

    #새로운 new_result 생성
    df_review_cut2.rename(columns={'review_result':'new_result'},inplace=True)
    df_review_cut2.reset_index(inplace=True)

    # 두가지 데이터 프레임 동일하게 정렬하고 합치기
    df_review_cut2.sort_values(by='name',inplace=True,ascending=False)
    df_review_cut = df_review_cut.drop_duplicates('name')
    df_review_cut.sort_values(by='name',ascending=False,inplace=True)
    df_review_cut['new_result'] = list(df_review_cut2['new_result'])

    #new result 순으로 정렬
    df_review_cut.sort_values(by='new_result',ascending=False,inplace=True)
    return df_review_cut

In [ ]:
df_review_cut = sort_review(df)

In [ ]:
df_review_cut

## 가격 yes or no, 성분 yes or no

In [ ]:
price_flag = 0
ingredient_flag = 0

In [ ]:
df_review_cut['new_result']

In [ ]:
min_max_sc = MinMaxScaler()

In [ ]:
ddff = df_review_cut['new_result']

In [ ]:
#가격이 중요할 경우 가중치 부여
if question_1 == 'q1_price' :
    df['new_result'] = df.apply(lambda x : x['new_result']+( 0.01*(1- x['price_num_score'])), axis=1)

In [ ]:
#성분이 중요할 경우 가중치 부여
if question_2 == 'q2_ingre':
    df['new_result'] = df.apply(lambda x : x['new_result']+( 0.01*(1-x['danger_num_score'])), axis=1)

In [ ]:
df_review_cut.sort_values(by='new_result',ascending=False,inplace=True)

In [ ]:
df_review_cut[:8]['name']

In [2]:
df = pd.read_excel('cream_sentiment_r.xlsx')

In [3]:
# 버트를 통해 도출된 리뷰데이터에 대한 긍정/부정 판단값을 새로운 열에 추가하고 한 제품의 total 긍정리뷰 비율 계산.
def sentiment_rate(df,subject):
    df['sentiment_judge'] = df['sentiment'].apply(lambda x : 1 if x > 0.5 else 0)
    list_sent = []
    list_name = list(df['name'])
    for element in list_name :
        count = list(df[df['name'] == element]['sentiment_judge'].value_counts())
        if len(count) == 2:
            count = round(count[0]/(count[0] + count[1]),2)
            list_sent.append(count)
        else :
            list_sent.append(1)
    df['sentiment_judge_rate'] = list_sent
    
    df.to_excel(f'{subject}_sentiment_result.xlsx',index =False)
    
    return(df)

In [4]:
sentiment_rate(df,"cream")

,Unnamed: 0,brand,name,bottle,price_num,star,product_info,product_type,ingredient,danger_num,...,offer,소분류,대분류,crim_image_link,price_num_score,danger_num_score,product_id,sentiment,sentiment_judge,sentiment_judge_rate
0,0,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.992140,1,0.94
1,1,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.957897,1,0.94
2,2,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.995551,1,0.94
3,3,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.982905,1,0.94
4,4,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.972929,1,0.94
5,5,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.937819,1,0.94
6,6,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.994878,1,0.94
7,7,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.990389,1,0.94
8,8,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.288881,0,0.94
9,9,주미소,워터풀 히알루론산 크림,50g,24000,4.68,수부지를 위한 청량수분케어- 물방울 얹은 듯 가볍게 스며드는 수분크림- 7종 히알루...,보습 수분공급 유수분밸런스 저자극,"정제수Water;Aqua피부컨디셔닝제, 용제부틸렌글라이콜Butylene Glycol...",0,...,0,수분크림,크림,https://dn5hzapyfrpio.cloudfront.net/product/c...,0.059169,0.045455,1,0.992801,1,0.94
